# Rolling Update Tests

Check rolling updates function as expected.

In [ ]:
import json
import time

Before we get started we'd like to make sure that we're making all the changes in a new blank namespace of the name `seldon`

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Change Image

We'll want to try modifying an image and seeing how the rolling update performs the changes.

We'll first create the following model:

In [ ]:
%%writefile resources/fixed_v1.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 3

Now we can run that model and wait until it's released

In [ ]:
!kubectl apply -f resources/fixed_v1.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

Let's confirm that the state of the model is Available

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now we can modify the model by providing a new image name, using the following config file:

In [ ]:
%%writefile resources/fixed_v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 3

In [ ]:
!kubectl apply -f resources/fixed_v2.yaml

Now let's actually send a couple of requests to make sure that there are no failed requests as the rolling update is performed

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 3:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v1.yaml

## Change Replicas (no rolling update)

We'll want to try modifying number of replicas and no rolling update is needed.

We'll first create the following model:

In [ ]:
%%writefile resources/fixed_v1_rep2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 2

Now we can run that model and wait until it's released

In [ ]:
!kubectl apply -f resources/fixed_v1_rep2.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

Let's confirm that the state of the model is Available

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now we can modify the model by providing a new image name, using the following config file:

In [ ]:
%%writefile resources/fixed_v1_rep4.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 4

In [ ]:
!kubectl apply -f resources/fixed_v1_rep4.yaml

Now let's actually send a couple of requests to make sure that there are no failed requests as the rolling update is performed

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 4:
        break
    time.sleep(1)
print("Rollout Success")

Now downsize back to 2

In [ ]:
!kubectl apply -f resources/fixed_v1_rep2.yaml

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 2:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v1_rep2.yaml

## Separate Service Orchestrator

We can test that the rolling update works when we use the annotation that allows us to have the service orchestrator on a separate pod, namely `seldon.io/engine-separate-pod: "true"`, as per the config file below. Though in this case both the service orchestrator and model pod will be recreated.

In [ ]:
%%writefile resources/fixed_v1_sep.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  annotations:
    seldon.io/engine-separate-pod: "true"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 1

In [ ]:
!kubectl apply -f resources/fixed_v1_sep.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

We can wait until the pod is available before starting the rolling update.

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [ ]:
%%writefile resources/fixed_v2_sep.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  annotations:
    seldon.io/engine-separate-pod: "true"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 1

In [ ]:
!kubectl apply -f resources/fixed_v2_sep.yaml

And we can send requests to confirm that the rolling update is performed without interruptions

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 1:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v1_sep.yaml

## Two PodSpecs

We can test that the rolling update works when we have multiple podSpecs in our deployment and only does a rolling update the first pod (which also contains the service orchestrator)

In [ ]:
%%writefile resources/fixed_v1_2podspecs.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier1
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier1
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 1

In [ ]:
!kubectl apply -f resources/fixed_v1_2podspecs.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

We can wait until the pod is available before starting the rolling update.

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [ ]:
%%writefile resources/fixed_v2_2podspecs.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier1
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier1
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 1

In [ ]:
!kubectl apply -f resources/fixed_v2_2podspecs.yaml

And we can send requests to confirm that the rolling update is performed without interruptions

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 1:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v1_2podspecs.yaml

## Two Models

We can test that the rolling update works when we have two predictors / models in our deployment.

In [ ]:
%%writefile resources/fixed_v1_2models.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 3

In [ ]:
!kubectl apply -f resources/fixed_v1_2models.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

We can wait until the pod is available before starting the rolling update.

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [ ]:
%%writefile resources/fixed_v2_2models.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 3

In [ ]:
!kubectl apply -f resources/fixed_v2_2models.yaml

And we can send requests to confirm that the rolling update is performed without interruptions

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 3:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v2_2models.yaml

## Two Predictors

We can test that the rolling update works when we have two predictors in our deployment.

In [ ]:
%%writefile resources/fixed_v1_2predictors.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: a
    replicas: 3
    traffic: 50
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: b
    replicas: 1
    traffic: 50

In [ ]:
!kubectl apply -f resources/fixed_v1_2predictors.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

We can wait until the pod is available before starting the rolling update.

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [ ]:
%%writefile resources/fixed_v2_2predictors.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: a
    replicas: 3
    traffic: 50
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: b
    replicas: 1
    traffic: 50

In [ ]:
!kubectl apply -f resources/fixed_v2_2predictors.yaml

And we can send requests to confirm that the rolling update is performed without interruptions

In [ ]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 3:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v2_2predictors.yaml

## Model name changes

This will not do a rolling update but create a new deployment.

In [ ]:
%%writefile resources/fixed_v1.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 3

In [ ]:
!kubectl apply -f resources/fixed_v1.yaml

We can wait until the pod is available.

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=fixed \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state = !kubectl get sdep fixed -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

Now when we apply the update, we should see the change taking place, but there should not be an actual full rolling update triggered.

In [ ]:
%%writefile resources/fixed_v2_new_name.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier2
      type: MODEL
    name: default
    replicas: 3

In [ ]:
!kubectl apply -f resources/fixed_v2_new_name.yaml

In [ ]:
time.sleep(5)
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numItems = len(resources["items"])
    if numItems == 1:
        break
    time.sleep(1)
print("Rollout Success")

In [ ]:
!kubectl delete -f resources/fixed_v2_new_name.yaml